In [60]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from six.moves import xrange  # pylint: disable=redefined-builtin

#from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.python.framework import dtypes

import random
import pandas as pd
import tensorflow as tf

import datetime
from tensorflow.python.framework import dtypes
import numpy as np



### reading CSV row data

format:

>50K, <=50K. 

age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 

education-num: continuous. 

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married
AF-spouse. 

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 

sex: Female, Male. 

capital-gain: continuous. 

capital-loss: continuous. 

hours-per-week: continuous. 

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.



In [80]:



dfTrain = pd.read_csv('Data/adult.data' ,sep=', ' ,header=None, names=['age','workclass', 'fnlwgt','education', 'education_num','martial_status', 'occupation','relationship', 'race', 'sex','capital_gain','capital_loss', 'hours_per_week', 'native_country', 'target'] , na_values=['?'])
dfTrain.drop('fnlwgt', axis=1, inplace=True )
shape = dfTrain.shape
trainRows = shape[0]
print ("training: read {0} columns and {1} rows".format( shape[1], shape[0]))

dfTest = pd.read_csv('Data/adult.test' ,sep=', ' , skiprows=1 ,header=None, names=['age','workclass', 'fnlwgt','education', 'education_num','martial_status', 'occupation','relationship', 'race', 'sex','capital_gain','capital_loss', 'hours_per_week', 'native_country', 'target'] , na_values=['?'])
dfTest.drop('fnlwgt', axis=1, inplace=True )
shape = dfTest.shape
print ("testing: read {0} columns and {1} rows".format( shape[1], shape[0]))

dfOut = pd.DataFrame()

/Users/ybercovich/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


training: read 14 columns and 32561 rows
testing: read 14 columns and 16281 rows


/Users/ybercovich/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [81]:
dfJoind = pd.concat([dfTrain, dfTest])
dfJoind.shape

(48842, 14)

In [82]:
trainRows

32561

### change target columns "readmitted" to binary form
we want to detect readmittion in less than 30 days.
so if we have '<30' value, it is 1, and otherwise 0

In [83]:
def categorize(row):
    if '<=50K' in row.target :
        return 0
    elif '>50K' in row.target :
        return 1
    else:
        raise 'not valid value'

dfOut['target_cat'] = dfJoind.apply(categorize, axis=1)

dfJoind.drop(  columns=['target'], inplace=True)


### Change All nominal columns to binary columns

In [84]:
for colName in dfJoind.columns:
    if dfJoind[colName].dtype == object:
        newColName = "is_" + colName
        for val in dfJoind[colName].unique():
            if val is np.nan:
                dfOut[newColName + "_nan"] = np.where(dfJoind[colName].isna() == True, 1,0)
            else:
                dfOut[newColName + "_" + val] = np.where(dfJoind[colName] == val, 1,0)

        dfJoind.drop(columns=[colName], inplace=True)

dfJoind.head(20)


,age,education_num,capital_gain,capital_loss,hours_per_week
0,39,13,2174,0,40
1,50,13,0,0,13
2,38,9,0,0,40
3,53,7,0,0,40
4,28,13,0,0,40
5,37,14,0,0,40
6,49,5,0,0,16
7,52,9,0,0,45
8,31,14,14084,0,50
9,42,13,5178,0,40


In [85]:
dfOut.head(10)

,target_cat,is_workclass_State-gov,is_workclass_Self-emp-not-inc,is_workclass_Private,is_workclass_Federal-gov,is_workclass_Local-gov,is_workclass_nan,is_workclass_Self-emp-inc,is_workclass_Without-pay,is_workclass_Never-worked,...,is_native_country_Outlying-US(Guam-USVI-etc),is_native_country_Scotland,is_native_country_Trinadad&Tobago,is_native_country_Greece,is_native_country_Nicaragua,is_native_country_Vietnam,is_native_country_Hong,is_native_country_Ireland,is_native_country_Hungary,is_native_country_Holand-Netherlands
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
dfOut = pd.concat( [dfOut,  (dfJoind - dfJoind.min())/ (dfJoind.max()-dfJoind.min())], axis=1 )

In [87]:
dfOut.shape

(48842, 108)

In [88]:
dfOut.head(20)

,target_cat,is_workclass_State-gov,is_workclass_Self-emp-not-inc,is_workclass_Private,is_workclass_Federal-gov,is_workclass_Local-gov,is_workclass_nan,is_workclass_Self-emp-inc,is_workclass_Without-pay,is_workclass_Never-worked,...,is_native_country_Vietnam,is_native_country_Hong,is_native_country_Ireland,is_native_country_Hungary,is_native_country_Holand-Netherlands,age,education_num,capital_gain,capital_loss,hours_per_week
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.301370,0.800000,0.021740,0.0,0.397959
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.452055,0.800000,0.000000,0.0,0.122449
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0.287671,0.533333,0.000000,0.0,0.397959
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0.493151,0.400000,0.000000,0.0,0.397959
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0.150685,0.800000,0.000000,0.0,0.397959
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0.273973,0.866667,0.000000,0.0,0.397959
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0.438356,0.266667,0.000000,0.0,0.153061
7,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.479452,0.533333,0.000000,0.0,0.448980
8,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0.191781,0.866667,0.140841,0.0,0.500000
9,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0.342466,0.800000,0.051781,0.0,0.397959


### split to test and train


In [89]:
trainRows

32561

In [91]:
trainDF_ready = dfOut.iloc[:trainRows,:]
testDF_ready  = dfOut.iloc[trainRows:,:]

print ("train:", trainDF_ready.shape)
print ("test:", testDF_ready.shape)
#dfOut = dfOut.sample(frac=1).reset_index(drop=True)


train: (32561, 108)
test: (16281, 108)


In [70]:
#dfOut.groupby('target_cat').count()

In [92]:

trainDF_ready.to_csv(columns=trainDF_ready.columns, sep='\t',path_or_buf='Data/adult_data_train.csv', index=False)
testDF_ready.to_csv(columns=testDF_ready.columns, sep='\t',path_or_buf='Data/adult_data_test.csv', index=False)
